In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1")
    .config("spark.sql.streaming.checkpointLocation", "checkpoints")
    .getOrCreate()
)

In [2]:
input_topic = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "input_topic")
    .load()
)

In [12]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

transformations = (
#    input_topic.selectExpr("value || value AS value")
    input_topic.groupby().count()
    .selectExpr("CAST(count AS String) AS value")
)

In [16]:
job = (
    transformations
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .outputMode("complete")
    .option("topic", "output_topic")
    .start()
)

In [17]:
job.stop()